<a href="https://colab.research.google.com/github/LuizFelipeLemon/machine_learning/blob/master/tensorflow_learning/regression02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a simple TensorFlow estimator
Let us build a simple TensorFlow estimator with a simple dataset for a multiple regression problem. We continue with the home price prediction, but now have two features, that is, we are considering two independent variables: the area of the house and its type (bungalow or apartment) on which we presume our price should depend:

We import the necessary modules. We will need TensorFlow and its feature_column module. Since our dataset contains both numeric and categorical data, we need the functions to process both types of data:

In [0]:
import tensorflow as tf
from tensorflow import feature_column as fc
numeric_column = fc.numeric_column 
categorical_column_with_vocabulary_list = fc.categorical_column_with_vocabulary_list

Now, we define the feature columns we will be using to train the regressor. Our dataset, as we mentioned, consists of two features "area" a numeric value signifying the area of the house and "type" telling if it is a "bungalow" or "apartment":


In [0]:
featcols = [
tf.feature_column.numeric_column("area"),
tf.feature_column.categorical_column_with_vocabulary_list("type",["bungalow","apartment"])
]

In the next step, we define an input function to provide input for training. The function returns a tuple containing features and labels:

In [0]:
def train_input_fn():
        features = {"area":[1000,2000,4000,1000,2000,4000],
             "type":["bungalow","bungalow","house",
                     "apartment","apartment","apartment"]}
        labels = [ 500 , 1000 , 1500 , 700 , 1300 , 1900 ]
        return features, labels

Next, we use the premade LinearRegressor estimator and fit it on the training dataset:

In [0]:
model = tf.estimator.LinearRegressor(featcols)
model.train(train_input_fn, steps=200)

Now that the estimator is trained, let us see the result of the prediction:

In [9]:
def predict_input_fn():
    features = {"area":[1500,1800],
                "type":["house","apt"]}
    return features
predictions = model.predict(predict_input_fn)
print(next(predictions))
print(next(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmph9qu58qj/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([692.7829], dtype=float32)}
{'predictions': array([830.9035], dtype=float32)}


# TensorFlow estimator in a real dataset

Now that we have the basics covered, let us apply these concepts to a real dataset. We will consider the Boston housing price dataset (http://lib.stat.cmu.edu/datasets/boston) collected by Harrison and Rubinfield in 1978. The dataset contains 506 sample cases. Each house is assigned 14 attributes:


In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow.feature_column as fc
from tensorflow.keras.datasets import boston_housing

Download the dataset:


In [0]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

Now let us define the features in our data, and for easy processing and visualization convert it into pandas DataFrame:

In [16]:
features = ['CRIM', 'ZN', 
            'INDUS','CHAS','NOX','RM','AGE',
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
x_train_df = pd.DataFrame(x_train, columns= features)
x_test_df = pd.DataFrame(x_test, columns= features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])
x_train_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


At present we are taking all the features; we suggest that you check the correlation among different features and the predicted label MEDV to choose the best features and repeat the experiment:

In [0]:
feature_columns = []
for feature_name in features:
        feature_columns.append(fc.numeric_column(feature_name, dtype=tf.float32))

We create the input function for the estimator. The function returns the tf.Data.Dataset object with a tuple: features and labels in batches. Use it to create train_input_fn and val_input_fn:

In [0]:
def estimator_input_fn(df_data, df_label, epochs=20, shuffle=True, batch_size=64):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function

train_input_fn = estimator_input_fn(x_train_df, y_train_df)
val_input_fn = estimator_input_fn(x_test_df, y_test_df, epochs=1, shuffle=False)

Next we instantiate a LinearRegressor estimator; we train it using training data using train_input_fn, and find the result for the validation dataset by evaluating the trained model using val_input_fn:

In [24]:
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir = 'logs/func/')
linear_est.train(train_input_fn, steps=100)
result = linear_est.evaluate(val_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'logs/func/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in 

Let's make a prediction on it:

In [23]:
result = linear_est.predict(val_input_fn)
for pred,exp in zip(result, y_test[:32]):
    print("Predicted Value: ", pred['predictions'][0], "Expected: ", exp)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpk0dxsio6/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted Value:  3.336909 Expected:  7.2
Predicted Value:  23.038101 Expected:  18.8
Predicted Value:  21.667313 Expected:  19.0
Predicted Value:  22.299356 Expected:  27.0
Predicted Value:  22.249353 Expected:  22.2
Predicted Value:  21.49